### Import all libraries

In [10]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys

import numpy as np

### Data pre-processing

In [11]:
class GAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = X_train / 127.5 - 1.
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("images/%d.png" % epoch)
        plt.close()


### Set the latent space dimension

In [13]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 512)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 256)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

99 [D loss: 0.012345, acc.: 100.00%] [G loss: 4.679221]
100 [D loss: 0.010723, acc.: 100.00%] [G loss: 4.663456]
101 [D loss: 0.009607, acc.: 100.00%] [G loss: 4.665903]
102 [D loss: 0.012544, acc.: 100.00%] [G loss: 4.841747]
103 [D loss: 0.010834, acc.: 100.00%] [G loss: 4.649850]
104 [D loss: 0.015076, acc.: 100.00%] [G loss: 4.637744]
105 [D loss: 0.014994, acc.: 100.00%] [G loss: 4.674530]
106 [D loss: 0.010952, acc.: 100.00%] [G loss: 4.832444]
107 [D loss: 0.010724, acc.: 100.00%] [G loss: 4.831974]
108 [D loss: 0.014084, acc.: 100.00%] [G loss: 4.755794]
109 [D loss: 0.011802, acc.: 100.00%] [G loss: 4.750478]
110 [D loss: 0.029409, acc.: 100.00%] [G loss: 4.329157]
111 [D loss: 0.014618, acc.: 100.00%] [G loss: 4.672868]
112 [D loss: 0.008000, acc.: 100.00%] [G loss: 4.778109]
113 [D loss: 0.019925, acc.: 100.00%] [G loss: 4.881740]
114 [D loss: 0.020336, acc.: 100.00%] [G loss: 4.398438]
115 [D loss: 0.029320, acc.: 98.44%] [G loss: 4.975776]
116 [D loss: 0.027074, acc.: 100.

250 [D loss: 0.841864, acc.: 43.75%] [G loss: 1.405969]
251 [D loss: 0.414880, acc.: 79.69%] [G loss: 2.244071]
252 [D loss: 0.919977, acc.: 37.50%] [G loss: 1.313168]
253 [D loss: 0.442908, acc.: 71.88%] [G loss: 2.318856]
254 [D loss: 0.833297, acc.: 37.50%] [G loss: 1.196849]
255 [D loss: 0.580354, acc.: 65.62%] [G loss: 1.695299]
256 [D loss: 0.680330, acc.: 57.81%] [G loss: 1.724695]
257 [D loss: 0.742601, acc.: 56.25%] [G loss: 1.232222]
258 [D loss: 0.532004, acc.: 64.06%] [G loss: 1.782948]
259 [D loss: 0.742385, acc.: 43.75%] [G loss: 1.372704]
260 [D loss: 0.724698, acc.: 50.00%] [G loss: 1.449874]
261 [D loss: 0.680896, acc.: 53.12%] [G loss: 1.388756]
262 [D loss: 0.596969, acc.: 68.75%] [G loss: 1.539911]
263 [D loss: 0.743031, acc.: 46.88%] [G loss: 1.176630]
264 [D loss: 0.703744, acc.: 51.56%] [G loss: 1.227219]
265 [D loss: 0.664615, acc.: 57.81%] [G loss: 1.452733]
266 [D loss: 0.780776, acc.: 48.44%] [G loss: 1.119648]
267 [D loss: 0.787584, acc.: 43.75%] [G loss: 1.

400 [D loss: 0.672374, acc.: 50.00%] [G loss: 0.688781]
401 [D loss: 0.640966, acc.: 60.94%] [G loss: 0.714767]
402 [D loss: 0.636941, acc.: 53.12%] [G loss: 0.728205]
403 [D loss: 0.644239, acc.: 59.38%] [G loss: 0.718098]
404 [D loss: 0.638767, acc.: 57.81%] [G loss: 0.698285]
405 [D loss: 0.629491, acc.: 51.56%] [G loss: 0.705125]
406 [D loss: 0.644538, acc.: 46.88%] [G loss: 0.707850]
407 [D loss: 0.645789, acc.: 56.25%] [G loss: 0.705209]
408 [D loss: 0.628211, acc.: 56.25%] [G loss: 0.703204]
409 [D loss: 0.636512, acc.: 54.69%] [G loss: 0.693051]
410 [D loss: 0.633963, acc.: 54.69%] [G loss: 0.720524]
411 [D loss: 0.635463, acc.: 53.12%] [G loss: 0.703252]
412 [D loss: 0.644388, acc.: 53.12%] [G loss: 0.683603]
413 [D loss: 0.624354, acc.: 59.38%] [G loss: 0.697209]
414 [D loss: 0.643138, acc.: 53.12%] [G loss: 0.686543]
415 [D loss: 0.636225, acc.: 51.56%] [G loss: 0.698194]
416 [D loss: 0.621040, acc.: 56.25%] [G loss: 0.712506]
417 [D loss: 0.615806, acc.: 56.25%] [G loss: 0.

547 [D loss: 0.626363, acc.: 57.81%] [G loss: 0.719500]
548 [D loss: 0.611543, acc.: 64.06%] [G loss: 0.734006]
549 [D loss: 0.650399, acc.: 59.38%] [G loss: 0.686273]
550 [D loss: 0.652241, acc.: 65.62%] [G loss: 0.714812]
551 [D loss: 0.657236, acc.: 56.25%] [G loss: 0.728755]
552 [D loss: 0.640012, acc.: 62.50%] [G loss: 0.687669]
553 [D loss: 0.633624, acc.: 62.50%] [G loss: 0.732387]
554 [D loss: 0.631968, acc.: 57.81%] [G loss: 0.745774]
555 [D loss: 0.618901, acc.: 68.75%] [G loss: 0.746359]
556 [D loss: 0.577420, acc.: 78.12%] [G loss: 0.735134]
557 [D loss: 0.612987, acc.: 67.19%] [G loss: 0.767947]
558 [D loss: 0.662953, acc.: 50.00%] [G loss: 0.768501]
559 [D loss: 0.647648, acc.: 54.69%] [G loss: 0.750432]
560 [D loss: 0.648706, acc.: 62.50%] [G loss: 0.731791]
561 [D loss: 0.647877, acc.: 59.38%] [G loss: 0.754689]
562 [D loss: 0.622610, acc.: 59.38%] [G loss: 0.737673]
563 [D loss: 0.638635, acc.: 56.25%] [G loss: 0.759143]
564 [D loss: 0.660775, acc.: 56.25%] [G loss: 0.

...

29347 [D loss: 0.624275, acc.: 64.06%] [G loss: 0.936929]
29348 [D loss: 0.653128, acc.: 59.38%] [G loss: 1.148861]
29349 [D loss: 0.636515, acc.: 67.19%] [G loss: 1.104468]
29350 [D loss: 0.639414, acc.: 70.31%] [G loss: 1.121459]
29351 [D loss: 0.761753, acc.: 50.00%] [G loss: 1.121488]
29352 [D loss: 0.612538, acc.: 64.06%] [G loss: 1.066535]
29353 [D loss: 0.676743, acc.: 64.06%] [G loss: 1.038484]
29354 [D loss: 0.545223, acc.: 82.81%] [G loss: 1.148578]
29355 [D loss: 0.662996, acc.: 64.06%] [G loss: 1.115612]
29356 [D loss: 0.555794, acc.: 75.00%] [G loss: 0.978997]
29357 [D loss: 0.590117, acc.: 68.75%] [G loss: 0.955183]
29358 [D loss: 0.630842, acc.: 67.19%] [G loss: 1.040765]
29359 [D loss: 0.568662, acc.: 65.62%] [G loss: 1.092686]
29360 [D loss: 0.604114, acc.: 64.06%] [G loss: 0.937494]
29361 [D loss: 0.603354, acc.: 70.31%] [G loss: 1.068823]
29362 [D loss: 0.638182, acc.: 62.50%] [G loss: 0.905885]
29363 [D loss: 0.616814, acc.: 76.56%] [G loss: 0.993972]
29364 [D loss:

29493 [D loss: 0.635063, acc.: 56.25%] [G loss: 1.084339]
29494 [D loss: 0.606320, acc.: 67.19%] [G loss: 1.219657]
29495 [D loss: 0.601746, acc.: 70.31%] [G loss: 1.125195]
29496 [D loss: 0.592784, acc.: 75.00%] [G loss: 1.088972]
29497 [D loss: 0.668961, acc.: 60.94%] [G loss: 1.094160]
29498 [D loss: 0.683816, acc.: 57.81%] [G loss: 0.999048]
29499 [D loss: 0.592651, acc.: 71.88%] [G loss: 1.049124]
29500 [D loss: 0.607005, acc.: 67.19%] [G loss: 0.942428]
29501 [D loss: 0.632839, acc.: 70.31%] [G loss: 1.040659]
29502 [D loss: 0.552371, acc.: 76.56%] [G loss: 1.034382]
29503 [D loss: 0.603172, acc.: 71.88%] [G loss: 1.050239]
29504 [D loss: 0.593728, acc.: 70.31%] [G loss: 1.015794]
29505 [D loss: 0.625184, acc.: 65.62%] [G loss: 0.972389]
29506 [D loss: 0.693693, acc.: 56.25%] [G loss: 1.064839]
29507 [D loss: 0.624370, acc.: 67.19%] [G loss: 1.090813]
29508 [D loss: 0.610604, acc.: 70.31%] [G loss: 1.015093]
29509 [D loss: 0.579721, acc.: 71.88%] [G loss: 1.080287]
29510 [D loss:

29639 [D loss: 0.625954, acc.: 67.19%] [G loss: 0.956729]
29640 [D loss: 0.559187, acc.: 64.06%] [G loss: 1.115472]
29641 [D loss: 0.576645, acc.: 70.31%] [G loss: 1.117123]
29642 [D loss: 0.635557, acc.: 54.69%] [G loss: 1.028879]
29643 [D loss: 0.756208, acc.: 54.69%] [G loss: 1.031512]
29644 [D loss: 0.683268, acc.: 56.25%] [G loss: 0.987743]
29645 [D loss: 0.668826, acc.: 54.69%] [G loss: 1.149450]
29646 [D loss: 0.583325, acc.: 67.19%] [G loss: 0.849334]
29647 [D loss: 0.625717, acc.: 60.94%] [G loss: 1.056932]
29648 [D loss: 0.686997, acc.: 53.12%] [G loss: 1.118834]
29649 [D loss: 0.653754, acc.: 64.06%] [G loss: 1.051986]
29650 [D loss: 0.586158, acc.: 70.31%] [G loss: 1.115646]
29651 [D loss: 0.685259, acc.: 60.94%] [G loss: 0.954607]
29652 [D loss: 0.624229, acc.: 67.19%] [G loss: 0.980578]
29653 [D loss: 0.681583, acc.: 56.25%] [G loss: 0.964265]
29654 [D loss: 0.655835, acc.: 59.38%] [G loss: 1.089956]
29655 [D loss: 0.508567, acc.: 76.56%] [G loss: 0.963696]
29656 [D loss:

29783 [D loss: 0.557816, acc.: 68.75%] [G loss: 1.038364]
29784 [D loss: 0.591442, acc.: 75.00%] [G loss: 0.980401]
29785 [D loss: 0.648268, acc.: 57.81%] [G loss: 1.069686]
29786 [D loss: 0.663376, acc.: 62.50%] [G loss: 1.006307]
29787 [D loss: 0.617396, acc.: 62.50%] [G loss: 1.059346]
29788 [D loss: 0.609027, acc.: 67.19%] [G loss: 1.247072]
29789 [D loss: 0.628755, acc.: 62.50%] [G loss: 1.119453]
29790 [D loss: 0.679044, acc.: 51.56%] [G loss: 0.976808]
29791 [D loss: 0.650293, acc.: 57.81%] [G loss: 1.113441]
29792 [D loss: 0.543067, acc.: 78.12%] [G loss: 0.964146]
29793 [D loss: 0.627110, acc.: 60.94%] [G loss: 0.983062]
29794 [D loss: 0.569777, acc.: 68.75%] [G loss: 1.002813]
29795 [D loss: 0.648470, acc.: 59.38%] [G loss: 0.968908]
29796 [D loss: 0.687741, acc.: 60.94%] [G loss: 1.080892]
29797 [D loss: 0.622486, acc.: 65.62%] [G loss: 1.011702]
29798 [D loss: 0.659673, acc.: 62.50%] [G loss: 1.066674]
29799 [D loss: 0.639670, acc.: 64.06%] [G loss: 1.051417]
29800 [D loss:

29928 [D loss: 0.694152, acc.: 59.38%] [G loss: 1.019367]
29929 [D loss: 0.603499, acc.: 73.44%] [G loss: 1.129490]
29930 [D loss: 0.579897, acc.: 68.75%] [G loss: 1.071374]
29931 [D loss: 0.610626, acc.: 59.38%] [G loss: 1.001339]
29932 [D loss: 0.559174, acc.: 70.31%] [G loss: 1.012186]
29933 [D loss: 0.668430, acc.: 65.62%] [G loss: 0.964734]
29934 [D loss: 0.602916, acc.: 64.06%] [G loss: 0.989044]
29935 [D loss: 0.626314, acc.: 60.94%] [G loss: 1.042839]
29936 [D loss: 0.651872, acc.: 60.94%] [G loss: 0.981364]
29937 [D loss: 0.717228, acc.: 51.56%] [G loss: 0.928403]
29938 [D loss: 0.656562, acc.: 67.19%] [G loss: 1.071749]
29939 [D loss: 0.562825, acc.: 71.88%] [G loss: 1.055072]
29940 [D loss: 0.651072, acc.: 56.25%] [G loss: 1.059448]
29941 [D loss: 0.594727, acc.: 71.88%] [G loss: 1.110333]
29942 [D loss: 0.602682, acc.: 70.31%] [G loss: 0.940526]
29943 [D loss: 0.692528, acc.: 50.00%] [G loss: 1.059145]
29944 [D loss: 0.678468, acc.: 59.38%] [G loss: 0.896993]
29945 [D loss: